In [2]:
%pip install tensorflow_addons

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys
sys.path.append("/content/drive/MyDrive/HM/")

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense, Dropout, BatchNormalization, Concatenate
import numpy as np
import pandas as pd
import gc

In [6]:
import tensorflow_addons as tfa

In [8]:
from src.data import DataHelper
from src.data.metrics import map_at_k

In [9]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [10]:
from pathlib import Path
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [11]:
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [13]:
RANK_EMBEDDING_DIM = 64
BATCH_SIZE = 2**12
NEPOCH = 20

In [14]:
TRAIN_WEEK_NUM = 4
WEEK_NUM = TRAIN_WEEK_NUM + 2

VERSION_NAME = "Recall 1"

In [15]:
data_dir = Path("/content/drive/MyDrive/HM/data/")
model_dir = Path("/content/drive/MyDrive/HM/models/")

In [16]:
dh = DataHelper(data_dir)
data = dh.load_data(name="encoded_full")

In [17]:
inter = data['inter']
inter = inter.loc[(inter.t_dat <= "2020-08-19")]

## Calculate & Load Embeddings

In [18]:
# article description - TFIDF - SVD
if not os.path.exists(data_dir/'external/tfidf_item_embd.npy'):
    articles = pd.read_csv(data_dir/'raw/articles.csv')

    corpus = articles[[col for col in articles.columns if 'name' in col] + ['detail_desc']].T.apply(lambda x: ' '.join(map(str,x))).T

    vectorizer = TfidfVectorizer(min_df=3)
    X = vectorizer.fit_transform(corpus)
    svd = TruncatedSVD(n_components=256, random_state=0)
    tfidf_item = svd.fit_transform(X)
    tfidf_item = np.concatenate([np.ones((1,256)), tfidf_item], axis=0)
    tfidf_item.dump(data_dir/'external/tfidf_item_embd.npy')
else:
    tfidf_item = np.load(data_dir/'external/tfidf_item_embd.npy', allow_pickle=True)

In [19]:
# article_id - customer_id TFIDF + SVD
if not os.path.exists(data_dir/'external/tfidf_item_embd2.npy'):
    corpus = inter.groupby('article_id').customer_id.apply(lambda x: ' '.join(map(str, x)))
    article_ids = np.array(list(corpus.index))

    vectorizer = TfidfVectorizer(min_df=3)
    X = vectorizer.fit_transform(corpus)
    svd = TruncatedSVD(n_components=128, random_state=0)
    X_svd = svd.fit_transform(X)

    item_num = data['item']['article_id'].nunique()
    tfidf_item2 = np.ones((item_num+1, 128)) / 128
    for i,iid in enumerate(article_ids):
        tfidf_item2[iid,:] = X_svd[i,:]

    tfidf_item2.dump(data_dir/'external/tfidf_item_embd2.npy')
else:
    tfidf_item2 = np.load(data_dir/'external/tfidf_item_embd2.npy', allow_pickle=True)

In [20]:
# customer_id - product_code TFIDF + SVD
if not os.path.exists(data_dir/'external/tfidf_user_embd.npy'):
    inter = inter.merge(data['item'][['article_id','product_code']], on=['article_id'], how='left')
    corpus = inter.groupby('customer_id').product_code.apply(lambda x: ' '.join(map(str, x)))
    customer_ids = np.array(list(corpus.index))

    vectorizer = TfidfVectorizer(min_df=3)
    X = vectorizer.fit_transform(corpus)
    svd = TruncatedSVD(n_components=128, random_state=0)
    X_svd = svd.fit_transform(X)

    user_num = data['user']['customer_id'].nunique()
    tfidf_user = np.ones((user_num+1, 128)) / 128
    for i,uid in enumerate(customer_ids):
        tfidf_user[uid,:] = X_svd[i,:]

    tfidf_user.dump(data_dir/'external/tfidf_user_embd.npy')
else:
    tfidf_user = np.load(data_dir/'external/tfidf_user_embd.npy', allow_pickle=True)

In [21]:
# * Load pre-trained embeddings
w2v_user_embd = np.load(data_dir/'external'/'w2v_user_embd.npy', allow_pickle=True)
w2v_item_embd = np.load(data_dir/'external'/'w2v_item_embd.npy', allow_pickle=True)
w2v_product_embd = np.load(data_dir/'external'/'w2v_product_embd.npy', allow_pickle=True)
image_item_embd = np.load(data_dir/'external'/'image_embd.npy', allow_pickle=True)
w2v_sg_user_embd = np.load(data_dir/'external'/'w2v_skipgram_user_embd.npy', allow_pickle=True)
w2v_sg_item_embd = np.load(data_dir/'external'/'w2v_skipgram_item_embd.npy', allow_pickle=True)
w2v_sg_product_embd = np.load(data_dir/'external'/'w2v_skipgram_product_embd.npy', allow_pickle=True)

## Load Candidates & Prepare Data

In [22]:
candidates = {}
labels = {}
for i in tqdm(range(1, WEEK_NUM)):
    candidates[i] = pd.read_parquet(data_dir/"processed"/VERSION_NAME/f"week{i}_candidate.pqt")
    labels[i] = pd.read_parquet(data_dir/"processed"/VERSION_NAME/f"week{i}_label.pqt")

100%|██████████| 5/5 [00:08<00:00,  1.69s/it]


In [23]:
feats = [
    x
    for x in candidates[1].columns
    if x
    not in [
        "label",
        "sales_channel_id",
        "t_dat",
        "week",
        "wv_similarity",
    ]
]

ids = ["customer_id", "article_id", "product_code"]
dense_feats = [x for x in feats if x not in ids]

In [24]:
for f in tqdm(dense_feats):
    for i in range(1,WEEK_NUM):
        if f in candidates[i].columns:
            candidates[i][f] = candidates[i][f].astype('float16')

100%|██████████| 96/96 [00:31<00:00,  3.07it/s]


In [25]:
full_data = pd.concat([candidates[i] for i in range(1,WEEK_NUM)], ignore_index=True)


inter = data['inter']
inter = inter[inter['t_dat']<'2020-08-19'] # * start date of the last valid week
inter['week'] = (pd.to_datetime('2020-09-29') - pd.to_datetime(inter['t_dat'])).dt.days // 7
inter = inter.merge(data['item'][["article_id", "product_code"]], on="article_id", how="left")

tmp = inter.groupby('article_id').week.mean()
full_data['article_time_mean'] = full_data['article_id'].map(tmp)

tmp = inter.groupby('customer_id').week.nth(-1)
full_data['customer_id_last_time'] = full_data['customer_id'].map(tmp)

tmp = inter.groupby('customer_id').week.nth(0)
full_data['customer_id_first_time'] = full_data['customer_id'].map(tmp)

tmp = inter.groupby('customer_id').week.mean()
full_data['customer_id_time_mean'] = full_data['customer_id'].map(tmp)

full_data['customer_id_gap'] = full_data['customer_id_first_time'] - full_data['customer_id_last_time']
extra_feats = [
    'article_time_mean', 
    'customer_id_last_time', 
    'customer_id_first_time', 
    'customer_id_time_mean',
    'customer_id_gap'
]
feats += extra_feats
dense_feats += extra_feats

for f in extra_feats:
    full_data[f] = full_data[f].astype('float16')


full_data = full_data[feats+['week','label']]
gc.collect()


train = full_data[full_data['week']>1]
valid = full_data[full_data['week']==1]

In [26]:
del candidates
gc.collect()

50

In [27]:
feat_dim = {}
for feat in ids:
    if feat in data['user'].columns:
        feat_dim[feat] = int(data['user'][feat].max()) + 1
    elif feat in data['item'].columns:
        feat_dim[feat] = int(data['item'][feat].max()) + 1
    else:
        feat_dim[feat] = int(full_data[feat].max()) + 1

In [28]:
del full_data
gc.collect()

100

In [29]:
X_train1 = train[['customer_id', 'article_id', 'product_code']].values.astype('int32')
X_train2 = np.zeros((X_train1.shape[0], len(dense_feats)), dtype='float32')
for i,f in tqdm(enumerate(dense_feats)):
    X_train2[:, i] = np.nan_to_num(train[f].values).astype('float32')
    del train[f]
y_train = train['label'].values

101it [00:32,  3.14it/s]


In [30]:
X_test1 = valid[['customer_id', 'article_id', 'product_code']].values.astype('int32')
X_test2 = np.zeros((X_test1.shape[0], len(dense_feats)), dtype='float32')
for i,f in tqdm(enumerate(dense_feats)):
    X_test2[:, i] = np.nan_to_num(valid[f].values).astype('float32')
    del valid[f]
y_test = valid['label'].values

101it [00:10, 10.08it/s]


## Train Model

In [31]:
customer_embd_layer_1 = Embedding(
    feat_dim["customer_id"], 128, weights=[w2v_sg_user_embd], trainable=False
)
customer_embd_layer_2 = Embedding(
    feat_dim["customer_id"], 128, weights=[w2v_user_embd], trainable=False
)
customer_embd_layer_3 = Embedding(
    feat_dim["customer_id"], 128, weights=[tfidf_user], trainable=False
)

In [32]:
article_embd_layer_1 = Embedding(
    feat_dim["article_id"], 128, weights=[w2v_sg_item_embd], trainable=False
)

article_embd_layer_2 = Embedding(
    feat_dim["article_id"], 128, weights=[w2v_item_embd], trainable=False
)

article_embd_layer_3 = Embedding(
    feat_dim["article_id"], 256, weights=[tfidf_item], trainable=False
)

article_embd_layer_4 = Embedding(
    feat_dim["article_id"], 128, weights=[tfidf_item2], trainable=False
)

article_embd_layer_5 = Embedding(
    feat_dim["article_id"], 512, weights=[image_item_embd], trainable=False
)

In [33]:
product_embd_layer_1 = Embedding(
    feat_dim["product_code"], 128, weights=[w2v_sg_product_embd], trainable=False
)
product_embd_layer_2 = Embedding(
    feat_dim["product_code"], 128, weights=[w2v_product_embd], trainable=False
)

In [34]:
inputs1 = Input(shape=X_train1.shape[1:], dtype=tf.int64)
inputs2 = Input(shape=X_train2.shape[1:], dtype=tf.float32)
input1 = tf.cast(inputs1, dtype=tf.int64)

x_c_id1 = customer_embd_layer_1(input1[:,0])
x_c_id2 = customer_embd_layer_2(input1[:,0])
x_c_id3 = customer_embd_layer_3(input1[:,0])

x_a_id1 = article_embd_layer_1(input1[:,1])
x_a_id2 = article_embd_layer_2(input1[:,1])
x_a_id3 = article_embd_layer_3(input1[:,1])
x_a_id3 = Dense(128)(x_a_id3)
x_a_id4 = article_embd_layer_4(input1[:,1])
x_a_id5 = article_embd_layer_5(input1[:,1])
x_a_id5 = Dense(128)(x_a_id5)

x_p_id1 = product_embd_layer_1(input1[:,2])
x_p_id2 = product_embd_layer_2(input1[:,2])


x_id = Concatenate(axis=-1)([
    x_c_id1, x_c_id2,
    x_a_id1, x_a_id2, x_a_id3, x_a_id4, x_a_id5,
    x_p_id1, x_p_id2,
])

x0 = Concatenate(axis=-1)([x_id, BatchNormalization()(inputs2)])

x = Dropout(0.3)(x0)
x = Dense(512, activation='swish')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='swish')(x)

x = Concatenate(axis=-1)([x, x0])
x = Dropout(0.3)(x)

output = Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=[inputs1, inputs2], outputs=[output])
# model.summary()
    
model.compile(
    tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=1e-4),
    loss = 'binary_crossentropy',
    metrics=['AUC']
)

In [35]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max')
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_dir/'small_nn.h5',
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True)

history = model.fit(
    [X_train1, X_train2], y_train.astype(int), 
    shuffle=True,
    batch_size=2048,
    validation_data=([X_test1, X_test2], y_test.astype(int)),
    epochs=30,
    callbacks=[checkpoint, early_stop]
)

In [36]:
model.load_weights(model_dir/'small_nn.h5')
probs = model.predict([X_test1, X_test2], batch_size=4096)
label = data['inter'][data['inter']['t_dat']>='2020-09-16']
label = label.groupby('customer_id')['article_id'].apply(list).reset_index()

valid['prob'] = probs
pred = valid.sort_values(by='prob',ascending=False).reset_index(drop=True)
pred = pred.groupby('customer_id')['article_id'].apply(list).reset_index()
pred.columns = ['customer_id','prediction']

label = label.merge(pred, on='customer_id', how='left')

map_at_k(label['article_id'], label['prediction'], k=12)

In [ ]:
valid = valid[['customer_id','article_id','prob']]
valid.to_parquet(data_dir/'external'/'small_nn_valid.pqt')

## Test

In [37]:
model.load_weights(model_dir/'small_nn.h5')

In [38]:
class TQDMPredictCallback(tf.keras.callbacks.Callback):
    def __init__(self, custom_tqdm_instance=None, tqdm_cls=tqdm, **tqdm_params):
        super().__init__()
        self.tqdm_cls = tqdm_cls
        self.tqdm_progress = None
        self.prev_predict_batch = None
        self.custom_tqdm_instance = custom_tqdm_instance
        self.tqdm_params = tqdm_params

    def on_predict_batch_begin(self, batch, logs=None):
        pass

    def on_predict_batch_end(self, batch, logs=None):
        self.tqdm_progress.update(batch - self.prev_predict_batch)
        self.prev_predict_batch = batch

    def on_predict_begin(self, logs=None):
        self.prev_predict_batch = 0
        if self.custom_tqdm_instance:
            self.tqdm_progress = self.custom_tqdm_instance
            return

        total = self.params.get('steps')
        if total:
            total -= 1

        self.tqdm_progress = self.tqdm_cls(total=total, **self.tqdm_params)

    def on_predict_end(self, logs=None):
        if self.tqdm_progress and not self.custom_tqdm_instance:
            self.tqdm_progress.close()

In [39]:
del train, valid, X_train1, X_train2, X_test1, X_test2
gc.collect()

694

In [ ]:
CHUNK_NUM = 2

In [40]:
for chunk in range(CHUNK_NUM):
    test_candidates = pd.read_parquet(data_dir/"processed"/VERSION_NAME/f"week0_candidate_{chunk}.pqt")

    tmp = inter.groupby('article_id').week.mean()
    test_candidates['article_time_mean'] = test_candidates['article_id'].map(tmp)

    tmp = inter.groupby('customer_id').week.nth(-1)
    test_candidates['customer_id_last_time'] = test_candidates['customer_id'].map(tmp)

    tmp = inter.groupby('customer_id').week.nth(0)
    test_candidates['customer_id_first_time'] = test_candidates['customer_id'].map(tmp)

    tmp = inter.groupby('customer_id').week.mean()
    test_candidates['customer_id_time_mean'] = test_candidates['customer_id'].map(tmp)

    test_candidates['customer_id_gap'] = test_candidates['customer_id_first_time'] - test_candidates['customer_id_last_time']

    for f in tqdm(dense_feats):
        test_candidates[f] = test_candidates[f].astype('float16')

    test1 = test_candidates[['customer_id', 'article_id', 'product_code']].values.astype('int32')
    test2 = np.zeros((test1.shape[0], len(dense_feats)), dtype='float32')
    for i,f in tqdm(enumerate(dense_feats)):
        test2[:, i] = np.nan_to_num(test_candidates[f].values).astype('float32')
        del test_candidates[f]
    gc.collect()

    probs = model.predict([test1, test2], batch_size=2048, callbacks=[TQDMPredictCallback()])
    test_candidates["prob"] = probs
    pred_nn = test_candidates[['customer_id','article_id','prob']]
    pred_nn.rename(columns={'article_id':'prediction'}, inplace=True)
    pred_nn['customer_id'] = pred_nn['customer_id'].astype(int)
    pred_nn.to_parquet(data_dir/'interim'/f'small_nn_test_{chunk}.pqt')

100%|██████████| 101/101 [01:21<00:00,  1.24it/s]
101it [01:32,  1.10it/s]
100%|██████████| 101/101 [01:27<00:00,  1.16it/s]
101it [01:31,  1.10it/s]
100%|██████████| 13503/13503 [03:25<00:00, 65.59it/s]


In [41]:
test_pred1 = pd.read_parquet(data_dir/'interim'/f'small_nn_test_0.pqt')
test_pred2 = pd.read_parquet(data_dir/'interim'/f'small_nn_test_1.pqt')

In [42]:
test_pred = pd.concat([test_pred1, test_pred2], ignore_index=True)
test_pred = test_pred.sort_values(by=["prob"], ascending=False).reset_index(drop=True)
test_pred = test_pred.drop_duplicates(['customer_id', 'prediction'], keep='first')

In [43]:
test_pred.to_parquet(data_dir/'processed'/'small_nn_test.pqt')